Escreva aqui o título do seu experimento
========================================



## Introdução



Esse código encontra um valor de mínimo global para a função de Himmelblau. Como uma primeira análise, pode ser interessante analisar algebricamente essa função, da seguinte forma: $f(x,y)= (x² + y - 11)² + (x+ y²- 7)²$

Os quatro pontos mínimos da função de Himmelblau estão localizados em (3, 2), (-2.805118, 3.131312), (-3.779310, -3.283186) e (3.584428, -1.848126)

## Objetivo



Sabendo dos diferentes pontos que podem ser encontrados para essa função, o objetivo da presente atividade consiste em encontrar, com auxílio de algoritmos genéticos, o ponto **MÍNIMO GLOBAL** para a função

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
# Bibliotecas necessárias para rodar as células abaixo!
# Para  abusca aleatória:
import random
from funcoes import populacao_blau
from funcoes import selecao_por_torneio_blau as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_blau
from funcoes import funcao_objetivo_blau
from funcoes import funcao_objetivo_pop_blau as funcao_objetivo_pop

## Códigos e discussão



Para resolver esse problema por meio de algoritmos genéticos, é necessário pensar o que eu consideraria como: Gene, Indivíduo, Função objetivo, Seleção, Mutação, Cruzamento.

Os `genes` serão valores de x e de y em um determinado intervalo de números. Logo, em cada `ìndivíduo` há dois genes apenas, cada um representando uma coordenada do ponto.

O que buscamos, no problema, é o ponto de menor valor da função em um determinado intervalo. Logo, a `função objetivo` deve retornar a projeção de cada ponto (indivíduo) na função $f(x,y)= (x² + y - 11)² + (x+ y²- 7)²$. Por fim, como é uma função de minimização, utilizaremos a mesma `seleção` utilizada no experimento 0.6 das senhas, assim como as mesmas funções de `mutação` e de `cruzamento`.



In [2]:
# constantes de busca
TAMANHO_POP = 12 
NUM_GERACOES = 100000 
CHANCE_MUTACAO = 0.02 # chance de ocorrer mutação em cada indivíduo durante cada geração
LIMITE = []
max_limite= 10
min_limite= -10
incremento= 0.01
while min_limite <= max_limite:
    # Adicionar o número à lista
    LIMITE.append(min_limite)
    # Incrementar o número pelo valor do incremento
    min_limite += incremento
    
CHANCE_DE_COMPETIR_IND = 0.5 # chance que cada indivíduo tem de ser chamado para o torneio
CHANCE_CRUZAMENTO = 0.5 # chance de ocorrer o cruzamento entre dois indivíduos
CHANCE_MUTACAO = 0.02
ERRO = 0.00001 # critério de parada

# constantes de problema
NUM_GENES = 2 # número de genes ou tamanho da senha
MINIMO = 0 # valor mínimo estimado para a função

In [3]:
# Funções Locais

def cria_populacao_inicial(tamanho, numero_genes):
    return populacao_blau(tamanho, numero_genes, LIMITE)

def funcao_mutacao(individuo):
    return mutacao_blau(individuo, LIMITE)

In [5]:
populacao = cria_populacao_inicial(TAMANHO_POP, NUM_GENES) # cria aleatoriamente uma população inicial

print('População inicial:') # mostra qual foi a população criada aleatoriamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind)
    
melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python

#for _ in range(NUM_GERACOES): # loop que começa a rodar cada geração
while melhor_fitness_ja_visto >= (MINIMO + ERRO):
   
    # Única alteração na estrutura do código
    populacao = funcao_selecao(populacao, CHANCE_DE_COMPETIR_IND) # Tem a função de sortear individuos na população e troca-los pelo melhor entre eles
    
    pais = populacao[0::2] # definição dos indivíduos que serão pais
    maes = populacao[1::2] # definição dos indivíduos que serão mães
    contador = 0 # estratégia para colocar os filhos no lugar dos pais
    for pai, mae in zip(pais, maes): # laço de repetição para pegar itens da lista de pais e mães
        if random.random() < CHANCE_CRUZAMENTO: # aplicando a possibilidade de cruzamento
            # vai acertar o cruzamento
            filho1, filho2 = funcao_cruzamento(pai, mae) # "calculando" o filho 1 e o filho 2
            populacao[contador] = filho1 # trocando o pai pelo filho 1
            populacao[contador + 1] = filho2 # trocando a mãe pelo filho 2
            
        contador = contador + 2 # atualização do contador
    
    for n in range(len(populacao)): #laço de repetição para mutação
        if random.random() <= CHANCE_MUTACAO: # chance de mutação
            individuo = populacao[n] # esxolhe o indivíduo
            populacao[n] = funcao_mutacao(individuo) # muta o indivíduo
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness
        
    
print()
print('População final:') # mostra qual foi a população final selecionada geneticamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind, funcao_objetivo_blau(ind))
print(melhor_individuo_ja_visto)
print(melhor_fitness_ja_visto)

População inicial:
Individuo  1 :  [1.329999999999832, -3.2200000000001445]
Individuo  2 :  [1.419999999999832, -6.100000000000083]
Individuo  3 :  [-0.9200000000001696, -5.350000000000099]
Individuo  4 :  [-8.57000000000003, 2.549999999999821]
Individuo  5 :  [1.299999999999832, 1.1699999999998318]
Individuo  6 :  [-5.1900000000001025, -2.9500000000001503]
Individuo  7 :  [0.29999999999983123, 8.069999999999704]
Individuo  8 :  [4.0899999999997885, -1.9400000000001705]
Individuo  9 :  [6.84999999999973, 9.499999999999673]
Individuo  10 :  [2.439999999999823, 1.1299999999998318]
Individuo  11 :  [5.659999999999755, -0.01000000000016889]
Individuo  12 :  [-5.980000000000086, -6.780000000000069]

População final:
Individuo  1 :  [2.9999999999998113, 2.019999999999832] 0.006864159999808442
Individuo  2 :  [2.9999999999998113, 2.019999999999832] 0.006864159999808442
Individuo  3 :  [2.9999999999998113, 2.019999999999832] 0.006864159999808442
Individuo  4 :  [2.9999999999998113, 2.019999999

In [8]:
lista= []
for _ in range (10):
    populacao = cria_populacao_inicial(TAMANHO_POP, NUM_GENES) # cria aleatoriamente uma população inicial

    print('População inicial:') # mostra qual foi a população criada aleatoriamente
    for i, ind in enumerate(populacao):
        print('Individuo ', i+1, ': ', ind)
    
    melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python

    #for _ in range(NUM_GERACOES): # loop que começa a rodar cada geração
    while melhor_fitness_ja_visto >= (MINIMO + ERRO):
   
        # Única alteração na estrutura do código
        populacao = funcao_selecao(populacao, CHANCE_DE_COMPETIR_IND) # Tem a função de sortear individuos na população e troca-los pelo melhor entre eles
    
        pais = populacao[0::2] # definição dos indivíduos que serão pais
        maes = populacao[1::2] # definição dos indivíduos que serão mães
        contador = 0 # estratégia para colocar os filhos no lugar dos pais
        for pai, mae in zip(pais, maes): # laço de repetição para pegar itens da lista de pais e mães
            if random.random() < CHANCE_CRUZAMENTO: # aplicando a possibilidade de cruzamento
                # vai acertar o cruzamento
                filho1, filho2 = funcao_cruzamento(pai, mae) # "calculando" o filho 1 e o filho 2
                populacao[contador] = filho1 # trocando o pai pelo filho 1
                populacao[contador + 1] = filho2 # trocando a mãe pelo filho 2
            
            contador = contador + 2 # atualização do contador
    
        for n in range(len(populacao)): #laço de repetição para mutação
            if random.random() <= CHANCE_MUTACAO: # chance de mutação
                individuo = populacao[n] # esxolhe o indivíduo
                populacao[n] = funcao_mutacao(individuo) # muta o indivíduo
            
        # melhor individuo já visto até agora
        fitness = funcao_objetivo_pop(populacao)
        menor_fitness = min(fitness)
        if menor_fitness < melhor_fitness_ja_visto:        
            posicao = fitness.index(menor_fitness)
            melhor_individuo_ja_visto = populacao[posicao]
            melhor_fitness_ja_visto = menor_fitness
    lista.append(melhor_individuo_ja_visto)
print(lista)

População inicial:
Individuo  1 :  [6.309999999999741, 0.8899999999998317]
Individuo  2 :  [4.109999999999788, -5.480000000000096]
Individuo  3 :  [1.329999999999832, 2.499999999999822]
Individuo  4 :  [-2.350000000000163, -2.6100000000001575]
Individuo  5 :  [8.579999999999693, -3.990000000000128]
Individuo  6 :  [-9.400000000000013, -6.540000000000074]
Individuo  7 :  [-4.4900000000001175, -6.990000000000064]
Individuo  8 :  [-3.7400000000001334, -7.140000000000061]
Individuo  9 :  [-4.450000000000118, 3.899999999999792]
Individuo  10 :  [-7.400000000000055, 8.299999999999699]
Individuo  11 :  [-4.530000000000117, 8.019999999999705]
Individuo  12 :  [7.509999999999716, 0.04999999999983112]
População inicial:
Individuo  1 :  [7.349999999999719, 6.219999999999743]
Individuo  2 :  [7.909999999999707, -4.000000000000128]
Individuo  3 :  [-5.150000000000103, 2.5199999999998215]
Individuo  4 :  [-9.950000000000001, -8.240000000000038]
Individuo  5 :  [2.3399999999998253, -7.550000000000052

## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.

